# Métricas de Clasificación

Los modelos de ML se basan en optimizar una función objetivo, buscando su mínimo o máximo.
- Es importante comprender que esta función objetivo generalmente está desacopla de la métrica de evaluación que queremos optimizar en la práctica.
- La función objetivo sirve como un proxy para la métrica de evaluación.

En una configuración de clasificación, el vector `objetivo` es categórica.

In [ ]:
import pandas as pd

blood_transfusion = pd.read_csv("../../../data/blood_transfusion.csv")
data = blood_transfusion.drop(columns="Class")
target = blood_transfusion["Class"]

Comencemos por verificar las clases presentes en el vector objetivo `target`.

In [ ]:
import matplotlib.pyplot as plt

target.value_counts().plot.barh()
plt.xlabel("Number of samples")
_ = plt.title("Number of samples per classes present\n in the target")

Podemos ver que el vector `target` contiene dos clases correspondientes a si un sujeto donó sangre.

Usaremos un clasificador de regresión logística para predecir este resultado.

Para centrarnos en la presentación de métricas, solo usaremos una sola división en lugar de validación cruzada.

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, shuffle=True, random_state=0, test_size=0.5
)

Usaremos un clasificador de regresión logística como modelo base. 

Entrenaremos el modelo en el conjunto de trenes y luego usaremos el conjunto de prueba para calcular la métrica de clasificación diferente.

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(data_train, target_train)

## Predicciones del clasificador

Antes de entrar en detalles sobre las métricas, recordaremos qué tipo de predicciones puede proporcionar un clasificador. 

Por esta razón, crearemos una muestra sintética para un nuevo donante potencial: donaron sangre dos veces en el pasado (1000 cm³ cada vez).La última vez fue hace 6 meses, y la primera vez se remonta a hace 20 meses.

In [ ]:
new_donor = pd.DataFrame(
    {
        "Recency": [6],
        "Frequency": [2],
        "Monetary": [1000],
        "Time": [20],
    }
)

Podemos obtener la clase predicha por el clasificador llamando al método `predict`.

In [ ]:
classifier.predict(new_donor)

Con esta información, nuestro clasificador predice que es más probable que este sujeto sintético no vuelva a donar sangre.

Sin embargo, no podemos verificar si la predicción es correcta (no conocemos el valor objetivo verdadero).

- Ese es el propósito del conjunto de pruebas.

Primero, predecimos si un sujeto dará sangre con la ayuda del clasificador entrenado.

In [ ]:
target_predicted = classifier.predict(data_test)
target_predicted[:5]

## Precisión como línea de base

Ahora que tenemos estas predicciones, podemos compararlas con las verdaderas predicciones (a veces llamadas *ground-truth*) que no hemos usado hasta ahora.

In [ ]:
target_test == target_predicted

En la comparación anterior, un valor `True` significa que el valor predicho por nuestro clasificador es idéntico al valor real, mientras que un`False` significa que nuestro clasificador cometió un error.

Una forma de obtener una tasa general que representa el rendimiento de generalización de nuestro clasificador sería calcular cuántas veces nuestro clasificador era correcto y dividirlo por el número de muestras en nuestro set.

In [ ]:
import numpy as np

np.mean(target_test == target_predicted)

Esta medida se llama **precisión**.

Aquí, nuestro clasificador es un 78% preciso para clasificar si un sujeto dará sangre.

`scikit-learn` proporciona una función que calcula esta métrica en el módulo` sklearn.metrics`.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(target_test, target_predicted)
print(f"Accuracy: {accuracy:.3f}")

`LogisticRegression` también tiene un método `score` (parte de la API estándar de Scikit-Learn), que calcula la puntuación de precisión.

In [ ]:
classifier.score(data_test, target_test)

## Confusion matrix and derived metrics

The comparison that we did above and the accuracy that we calculated did not
take into account the type of error our classifier was making. Accuracy is an
aggregate of the errors made by the classifier. We may be interested in finer
granularity - to know independently what the error is for each of the two
following cases:

- we predicted that a person will give blood but they did not;
- we predicted that a person will not give blood but they did.

## Matriz de confusión y métricas derivadas

La comparación que hicimos anteriormente y la precisión que calculamos no tuvo en cuenta el tipo de error que nuestro clasificador estaba cometiendo.

La precisión es un agregado de los errores cometidos por el clasificador.

Es posible que estemos interesados ​​en la granularidad más fina. Saber independientemente cuál es el error para cada uno de los dos casos siguientes:

- Predijimos que una persona dará sangre pero no lo hizo;
- Predijimos que una persona no dará sangre, pero lo hizo.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

_ = ConfusionMatrixDisplay.from_estimator(classifier, data_test, target_test)

The in-diagonal numbers are related to predictions that were correct while
off-diagonal numbers are related to incorrect predictions
(misclassifications). We now know the four types of correct and erroneous
predictions:

* the top left corner are true positives (TP) and corresponds to people who
  gave blood and were predicted as such by the classifier;
* the bottom right corner are true negatives (TN) and correspond to people who
  did not give blood and were predicted as such by the classifier;
* the top right corner are false negatives (FN) and correspond to people who
  gave blood but were predicted to not have given blood;
* the bottom left corner are false positives (FP) and correspond to people who
  did not give blood but were predicted to have given blood.

Once we have split this information, we can compute metrics to highlight the
generalization performance of our classifier in a particular setting. For
instance, we could be interested in the fraction of people who really gave
blood when the classifier predicted so or the fraction of people predicted to
have given blood out of the total population that actually did so.

The former metric, known as the precision, is defined as `TP / (TP + FP)` and
represents how likely the person actually gave blood when the classifier
predicted that they did. The latter, known as the recall, defined as
`TP / (TP + FN)` and assesses how well the classifier is able to correctly
identify people who did give blood. We could, similarly to accuracy,
manually compute these values, however scikit-learn provides functions to
compute these statistics.

Los números en diagonales están relacionados con predicciones que fueron correctas, mientras que los números fuera de la diagonal están relacionados con predicciones incorrectas (clasificaciones erróneas).Ahora conocemos los cuatro tipos de predicciones correctas y erróneas:

* La esquina superior izquierda son verdaderos positivos (TP) y corresponde a personas que dieron sangre y fueron predicho como tal por el clasificador;
* La esquina inferior derecha son verdaderos negativos (TN) y corresponden a personas que no dieron sangre y fueron predicho como tal por el clasificador;
* La esquina superior derecha son falsos negativos (FN) y corresponden a personas que dieron sangre pero se predijo que no habían dado sangre;
* La esquina inferior izquierda son falsos positivos (FP) y corresponden a personas que no dieron sangre pero se predijo que habían dado sangre.

Una vez que hemos dividido esta información, podemos calcular métricas para resaltar el rendimiento de generalización de nuestro clasificador en un entorno particular.Por ejemplo, podríamos estar interesados ​​en la fracción de personas que realmente dieron sangre cuando el clasificador predijo eso o la fracción de las personas que predijeron haber dado sangre de la población total que realmente lo hizo.

- La precisión, se define como `TP / (TP + FP)` y representa la probabilidad de que la persona realmente diera sangre cuando el clasificador predijo que sí.
- Este último, conocido como el recall, definido como `TP / (TP + FN)` y evalúa qué tan bien el clasificador puede identificar correctamente a las personas que dieron sangre.

Podríamos, de manera similar a la precisión, calcular manualmente estos valores, sin embargo, Scikit-Learn proporciona funciones para calcular estas estadísticas.

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(target_test, target_predicted, pos_label="donated")
recall = recall_score(target_test, target_predicted, pos_label="donated")

print(f"Precision score: {precision:.3f}")
print(f"Recall score: {recall:.3f}")

Estos resultados están en línea con lo que se vio en la matriz de confusión.
- Mirando la columna izquierda, más de la mitad de las predicciones "donadas" fueron correctas, lo que condujo a una precisión por encima de 0.5.
- Sin embargo, nuestro clasificador equipó mal a muchas personas que dieron sangre como "no donada", lo que llevó a un retiro muy bajo de alrededor de 0.1.

## El problema del desequilibrio de clases
En esta etapa, podríamos hacernos una pregunta razonable: Si bien la precisión no se veía mala (es decir, 77%), la puntuación de recuperación es relativamente baja (es decir, 12%).
- Como mencionamos, la precisión y el recall solo se centran en las muestras previstas para ser positivas, mientras que la precisión tiene en cuenta ambos.
- Además, no observamos la relación de clases (etiquetas).Podríamos verificar esta relación en el conjunto de entrenamiento.

In [ ]:
target_train.value_counts(normalize=True).plot.barh()
plt.xlabel("Class frequency")
_ = plt.title("Class frequency in the training set")

Observamos que la clase positiva, 'donated', comprende solo el 24% de las muestras. 

La buena precisión de nuestro clasificador se vincula a su capacidad para predecir correctamente la clase negativa `not donated` que puede o no ser relevante, dependiendo de la aplicación. 

Podemos ilustrar el problema usando un clasificador ficticio como línea de base.

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_classifier = DummyClassifier(strategy="most_frequent")
dummy_classifier.fit(data_train, target_train)
print(
    "Accuracy of the dummy classifier: "
    f"{dummy_classifier.score(data_test, target_test):.3f}"
)

Con el clasificador ficticio, que siempre predice la clase negativa `not donated`, obtenemos un puntaje de precisión del 76%. 
- Por lo tanto, significa que este clasificador, sin aprender nada de los datos `data`, es capaz de predecir con tanta precisión como nuestro modelo de regresión logística.

El problema ilustrado anteriormente también se conoce como el **problema de desequilibrio de clases**. 
- Cuando las clases están desequilibradas, la precisión no debe usarse. 
- En este caso, uno debe usar la precisión y el recall como se presentó anteriormente o la  *precisión equilibrada (balanced accuracy)* en lugar de la precisión.

In [ ]:
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy = balanced_accuracy_score(target_test, target_predicted)
print(f"Balanced accuracy: {balanced_accuracy:.3f}")

La *precisión equilibrada* es equivalente a la precisión en el contexto de clases equilibradas.Se define como el retiro promedio obtenido en cada clase.

## Evaluación y diferentes umbrales de probabilidad

Todas las estadísticas que presentamos ahora confían en `classifier.predict` que genera la etiqueta más probable.
- No hemos hecho uso de la probabilidad asociada con esta predicción, lo que da la confianza del clasificador en esta predicción.
- Por defecto, la predicción de un clasificador corresponde a un umbral de 0.5 probabilidad en un problema de clasificación binaria.

Podemos verificar rápidamente esta relación con el clasificador que entrenamos.

In [ ]:
target_proba_predicted = pd.DataFrame(
    classifier.predict_proba(data_test), columns=classifier.classes_
)
target_proba_predicted[:5]

In [ ]:
target_predicted = classifier.predict(data_test)
target_predicted[:5]

Dado que las probabilidades suman 1, podemos obtener la clase con la mayor probabilidad sin usar el umbral 0.5.

In [ ]:
equivalence_pred_proba = (
    target_proba_predicted.idxmax(axis=1).to_numpy() == target_predicted
)
np.all(equivalence_pred_proba)

El umbral de decisión predeterminado (0.5) podría no ser el mejor umbral que conduce a un rendimiento de generalización óptimo de nuestro clasificador. 
- En este caso, uno puede variar el umbral de decisión y, por lo tanto, la predicción subyacente, y calcular las mismas estadísticas presentadas anteriormente. 
- Por lo general, los dos métricos retiran y la precisión se calculan y se representan en un gráfico. 
- Cada métrica trazada en un eje de gráfico y cada punto en el gráfico corresponde a un umbral de decisión específico. 

Comencemos calculando la curva de recolección de precisión.

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

disp = PrecisionRecallDisplay.from_estimator(
    classifier, data_test, target_test, pos_label="donated", marker="+"
)
disp = PrecisionRecallDisplay.from_estimator(
    dummy_classifier,
    data_test,
    target_test,
    pos_label="donated",
    color="tab:orange",
    linestyle="--",
    ax=disp.ax_,
)
plt.xlabel("Recall (also known as TPR or sensitivity)")
plt.ylabel("Precision (also known as PPV)")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
_ = disp.ax_.set_title("Precision-recall curve")

> Tip: Scikit-Learn devolverá una pantalla que contenga todo el elemento de trazado.En particular, las pantallas expondrán un eje matplotlib, llamado `ax_`, que se puede usar para agregar un nuevo elemento en el eje.Puede consultar la documentación para tener más información sobre las visualizaciones en Scikit-Learn

En esta curva, cada cruz azul corresponde a un nivel de probabilidad que utilizamos como umbral de decisión.Podemos ver que, al variar este umbral de decisión, obtenemos diferentes valores de precisión frente a recuperación. 

Un clasificador perfecto tendría una precisión de 1 para todos los valores de recuperación.Una métrica que caracteriza la curva está vinculada al área bajo la curva (AUC) y se llama Precision promedio (AP).Con un clasificador ideal, la precisión promedio sería 1.

Observe que el AP de un `DummyClassifier`, utilizado como línea de base para definir el nivel de posibilidad, coincide con el número de muestras en la clase positiva dividida por el número total de muestras (este número se llama prevalencia de la clase positiva).

In [ ]:
prevalence = (
    target_test.value_counts()["donated"] / target_test.value_counts().sum()
)
print(f"Prevalence of the class 'donated': {prevalence:.2f}")

La métrica de precisión y recall se centra en la clase positiva, sin embargo, uno podría estar interesado en el compromiso entre discriminar con precisión la clase positiva y discriminar con precisión las clases negativas. 
- Las estadísticas utilizadas para esto son la *sensibilidad* y la *especificidad*. 
- La sensibilidad es solo otro nombre para el recall. 
- Sin embargo, la especificidad mide la proporción de muestras clasificadas correctamente en la clase negativa definida como: `TN / (TN + FP)`. 
- Similar a la curva de recolección de precisión, la sensibilidad y la especificidad generalmente se trazan como una curva llamada curva de características operativas del receptor (*ROC*).

A continuación se muestra una curva:

In [ ]:
from sklearn.metrics import RocCurveDisplay

disp = RocCurveDisplay.from_estimator(
    classifier, data_test, target_test, pos_label="donated", marker="+"
)
disp = RocCurveDisplay.from_estimator(
    dummy_classifier,
    data_test,
    target_test,
    pos_label="donated",
    color="tab:orange",
    linestyle="--",
    ax=disp.ax_,
)
plt.xlabel("False positive rate")
plt.ylabel("True positive rate\n(also known as sensitivity or recall)")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
_ = disp.ax_.set_title("Receiver Operating Characteristic curve")

Esta curva se construyó utilizando el mismo principio que la curva de recuperación de precisión: variamos el umbral de probabilidad para determinar la predicción "dura" y calcular las métricas. 
- Al igual que con la curva de recuperación de precisión, podemos calcular el área bajo el ROC (ROC-AUC) para caracterizar el rendimiento de generalización de nuestro clasificador. 
- Sin embargo, es importante observar que el límite inferior del ROC-AUC es 0.5. 
- De hecho, mostramos el rendimiento de generalización de un clasificador ficticio (la línea naranja discontinua) para mostrar que incluso el peor rendimiento de generalización obtenido estará por encima de esta línea.

En lugar de usar un clasificador ficticio, puede usar el parámetro `trapt_chance_level` disponible en las pantallas ROC y PR:

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(15, 7))

PrecisionRecallDisplay.from_estimator(
    classifier,
    data_test,
    target_test,
    pos_label="donated",
    marker="+",
    plot_chance_level=True,
    chance_level_kw={"color": "tab:orange", "linestyle": "--"},
    ax=axs[0],
)
RocCurveDisplay.from_estimator(
    classifier,
    data_test,
    target_test,
    pos_label="donated",
    marker="+",
    plot_chance_level=True,
    chance_level_kw={"color": "tab:orange", "linestyle": "--"},
    ax=axs[1],
)

_ = fig.suptitle("PR and ROC curves")